# RF Chain and Linearity Commissioning Test
copied from system health displays NB to be updated

## Table of Contents
#### [1 - Global Configuration](#1-global-configuration)
#### [2 - Setup Test Equipment](#2-test-equipment-setup)
#### [3 - Configure Scan Tests](#3-configure-scan-tests)
#### [4 - Debug Telescope](#4-debug-telescope)
#### [5 - Dashboard Links](#5-dashboard-links)
#### [6 - System Health Displays Tests](#6-system-health-displays-tests)

## 1 Global Configuration

### 1.1 Import dependencies

In [3]:
import sys

sys.path.append("../../src")

import json
import logging
import os
import pathlib
import time
import tango

import ska_ser_logging
from bokeh.io import output_notebook
from ska_oso_pdm.entities.common.target import (
    CrossScanParameters,
    FivePointParameters,
    RasterParameters,
    SinglePointParameters,
    StarRasterParameters,
)
from ska_oso_pdm.entities.sdp import BeamMapping
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions.devicecontrol.resource_control import get_request_json
from ska_oso_scripting.objects import SubArray, Telescope
from ska_tmc_cdm.messages.central_node.assign_resources import AssignResourcesRequest
from ska_tmc_cdm.messages.central_node.sdp import Channel
from ska_tmc_cdm.messages.subarray_node.configure import ConfigureRequest
from ska_tmc_cdm.messages.subarray_node.configure.core import ReceiverBand

from ska_mid_jupyter_notebooks.cluster.cluster import Environment, TangoDeployment
from ska_mid_jupyter_notebooks.dish.dish import TangoDishDeployment
from ska_mid_jupyter_notebooks.helpers.path import project_root
from ska_mid_jupyter_notebooks.obsconfig.config import ObservationSB
from ska_mid_jupyter_notebooks.obsconfig.target_spec import TargetSpec
from ska_mid_jupyter_notebooks.sut.rendering import TelescopeMononitorPlot
from ska_mid_jupyter_notebooks.sut.state import TelescopeDeviceModel, get_telescope_state
from ska_mid_jupyter_notebooks.sut.sut import TangoSUTDeployment, disable_qa
from ska_mid_jupyter_notebooks.test_equipment.rendering import get_test_equipment_monitor_plot
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment
print("OK")

OK


### 1.2 Setup Global Variables and Configuration

In [4]:
# print(f"DISH_LMC_IN_THE_LOOP={os.environ('DISH_LMC_IN_THE_LOOP')}")
debug_mode = True  # This setting enables printing of diagnostics
enable_logging = True  # This enables logging and sets the global log_level to debug
dishlmc_enabled = True  # Set this to true if you have a dish LMC deployment
executon_environment = Environment.Integration
branch_name = None  # Set this if you are using an on-demand deployment (i.e. Environment.CI)
if enable_logging:
    ska_ser_logging.configure_logging(logging.DEBUG)
test_equipment = TangoTestEquipment()
print(f"Test Equipment Configured: {test_equipment}\n")
# namespace_override parameter can be used to override auto-configured SUT namespace
sut_namespace_override = ""
subarray_count = 1
subarray_id = 1
sut = TangoSUTDeployment(
    branch_name,
    executon_environment,
    namespace_override=sut_namespace_override,
    subarray_index=subarray_id,
)
print(f"SUT configured: {str(sut)}\n")
dish_ids = ["0001", "0036"]
# namespace_override parameter can be used to override auto-configured dish namespace
dish_namespace_overrides = ["", ""]
if dishlmc_enabled:
    dish_deployments = []
    for i, d in enumerate(dish_ids):
        dish = TangoDishDeployment(
            f"ska{d[1:]}",
            branch_name=branch_name,
            environment=executon_environment,
            namespace_override=dish_namespace_overrides[i],
        )
        print(f"Dish {d} configured: {dish}\n")
        dish_deployments.append(dish)
else:
    dish_deployments = []

timestr = time.strftime("%Y%m%d-%H%M")
notebook_output_dir = pathlib.Path(
    project_root(), f"notebook-execution-data/configure_scan_for_commissioning/execution-{timestr}"
)
os.makedirs(notebook_output_dir)
# we disable qa as it is not been properly verified
disable_qa()
print("OK")

Test Equipment Configured: TangoTestEquipment{namespace=test-equipment; tango_host=tango-databaseds.test-equipment.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.test-equipment.svc.miditf.internal.skao.int:8765}

SUT configured: TangoSUTDeployment{subarray_index=1; namespace=integration; tango_host=tango-databaseds.integration.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.integration.svc.miditf.internal.skao.int:8765}

Dish 0001 configured: TangoDishDeployment{dish_id=ska001; namespace=integration-dish-lmc-ska001; tango_host=tango-databaseds.integration-dish-lmc-ska001.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.integration-dish-lmc-ska001.svc.miditf.internal.skao.int:8765}

Dish 0036 configured: TangoDishDeployment{dish_id=ska036; namespace=integration-dish-lmc-ska036; tango_host=tango-databaseds.i

### 1.3 Test Connections to Namespaces

In [5]:
sut.smoke_test()
test_equipment.smoke_test()
for dish_deployment in dish_deployments:
    dish_deployment.smoke_test()
print("OK")

CIA PingResponse (integration): {"result":"ok","time":"2024-05-10T15:16:09.626598"}
CIA PingResponse (test-equipment): {"result":"ok","time":"2024-05-10T15:16:10.085949"}
mid-itf/siggen/1 is reachable
mid-itf/progattenuator/1 is reachable
mid-itf/spectana/1 is reachable
mid-itf/skysimctl/4 is reachable
CIA PingResponse (integration-dish-lmc-ska001): {"result":"ok","time":"2024-05-10T15:16:11.558678"}
CIA PingResponse (integration-dish-lmc-ska036): {"result":"ok","time":"2024-05-10T15:16:12.122572"}
OK


### 1.4 Export System Configuration

In [6]:
deployment: TangoDeployment
for deployment in [sut, test_equipment, *dish_deployments]:
    deployment.export_chart_configuration(output_dir=notebook_output_dir)
print("OK")

Exporting configuration using http://config-inspector.integration.svc.miditf.internal.skao.int:8765
ReleaseResponse (integration): {
    "chart": "ska-mid-itf",
    "version": "22.0.1",
    "sub_charts": [
        {
            "chart": "ska-tango-base",
            "version": "0.4.9"
        },
        {
            "chart": "ska-tango-util",
            "version": "0.4.10"
        },
        {
            "chart": "ska-tmc-mid",
            "version": "0.19.1"
        },
        {
            "chart": "ska-csp-lmc-mid",
            "version": "0.18.2"
        },
        {
            "chart": "ska-mid-cbf-mcs",
            "version": "0.14.1"
        },
        {
            "chart": "ska-sdp",
            "version": "0.19.1"
        },
        {
            "chart": "ska-ser-config-inspector",
            "version": "0.2.2"
        },
        {
            "chart": "ska-tango-taranta",
            "version": "2.8.3"
        },
        {
            "chart": "ska-tango-tangogql",
   

## 2 Test Equipment Setup

Use the noise source at nominal levels for input to at least one SPFRx.  

### 2.1 Configure Test Equipment State 

In [7]:
test_equipment_state = get_equipment_model(test_equipment)
test_equipment.devices
print("OK")

OK


### 2.2 Print Test Equipment Diagnostics

In [8]:
test_equipment.print_diagnostics()
print("OK")

mid-itf/skysimctl/4 State: ON
mid-itf/skysimctl/4 Band: 1
mid-itf/skysimctl/4 Correlated_Noise_Source: True
mid-itf/skysimctl/4 Uncorrelated_Noise_Sources: False
mid-itf/skysimctl/4 H_Channel: False
mid-itf/skysimctl/4 V_Channel: False
mid-itf/skysimctl/4 temperature: 23.8
mid-itf/skysimctl/4 humidity: 0.0
mid-itf/siggen/1 versionId: 0.9.1
mid-itf/siggen/1 adminMode: 0
mid-itf/siggen/1 State: ON
mid-itf/siggen/1 healthState: HealthState.UNKNOWN
mid-itf/siggen/1 frequency: 800000000.0
mid-itf/siggen/1 power_cycled: False
mid-itf/siggen/1 power_dbm: -30.0
mid-itf/siggen/1 rf_output_on: True
mid-itf/siggen/1 controlMode: 0
mid-itf/siggen/1 simulationMode: 0
mid-itf/siggen/1 testMode: 0
mid-itf/siggen/1 loggingLevel: 4
mid-itf/siggen/1 command_error: False
mid-itf/siggen/1 device_error: False
mid-itf/siggen/1 execution_error: False
mid-itf/siggen/1 query_error: False
mid-itf/spectana/1 adminMode: 0
mid-itf/spectana/1 State: ON
mid-itf/spectana/1 attenuation: 15
mid-itf/spectana/1 frequency

mid-itf/progattenuator/1 channel_1: 10.0
mid-itf/progattenuator/1 controlMode: 0
mid-itf/progattenuator/1 healthState: HealthState.UNKNOWN
mid-itf/progattenuator/1 loggingLevel: 5
mid-itf/progattenuator/1 simulationMode: 0
mid-itf/progattenuator/1 testMode: 0
OK


### 2.3 Create Test Equipment Plot

In [9]:
monitor_plot = get_test_equipment_monitor_plot()
test_equipment_state.subscribe_to_test_equipment_state(monitor_plot.handle_device_state_change)
output_notebook()
monitor_plot.show()
test_equipment_state.activate()
print("OK")

Loading BokehJS ...

OK


### 2.4 Turn offline Test Equipment devices ONLINE

In [10]:
# set any offline devices to online
test_equipment.turn_online()
print("OK")

set mid-itf/siggen/1 adminMode already ONLINE
set mid-itf/progattenuator/1 adminMode already ONLINE
set mid-itf/spectana/1 adminMode already ONLINE
OK


### 2.5 Display Test Equipment Device States 

In [11]:
test_equipment_state.state["devices_states"]
print("OK")

OK


### 2.6 Configure Signal Generator and set noise

In [12]:
frequency_to_set = 800e6

signal_generator = test_equipment.signal_generator
print(f"Current signal generator frequency: {signal_generator.frequency}")
signal_generator.write_attribute("frequency", frequency_to_set)
time.sleep(1)
print(f"Updated signal generator frequency: {signal_generator.frequency}")
assert (
    signal_generator.frequency == frequency_to_set
), f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
print("OK")

Current signal generator frequency: 800000000.0
Updated signal generator frequency: 800000000.0
OK


## 3 Configure Scan Tests

Use OSO scripting interface to TMC to run the script for assigning resources for a single sub-array, configuring and running a  scan.  

### 3.1 Setup Telescope Monitoring

#### 3.1.1 Configure Telescope Monitoring

In [13]:
# setup monitoring
# use telescope state object for state monitoring
print(f"Monitor {subarray_count} subarrays with dish IDs {dish_ids}")
device_model = TelescopeDeviceModel(dish_ids, subarray_count)
try:
    print(f"Get telescope state for device model {device_model}: sysytem under test {sut}")
    telescope_state = get_telescope_state(device_model, sut)
except tango.DevFailed as terr:
    print(f"ERROR: {terr.args[0].desc.strip()}")
    telescope_state = None
telescope_monitor_plot = TelescopeMononitorPlot(plot_width=900, plot_height=200)
if telescope_state is not None:
    # use monitor plot as a dashboard
    # set up events to monitor
    telescope_state.subscribe_to_on_off(telescope_monitor_plot.observe_telescope_on_off)
    telescope_state.subscribe_to_subarray_resource_state(
        telescope_monitor_plot.observe_subarray_resources_state
    )
    telescope_state.subscribe_to_subarray_configurational_state(
        telescope_monitor_plot.observe_subarray_configuration_state
    )
    telescope_state.subscribe_to_subarray_scanning_state(
        telescope_monitor_plot.observe_subarray_scanning_state
    ) 
else:
    print("ERROR: telescope state is unknown")
output_notebook()
print("OK")

Monitor 1 subarrays with dish IDs ['0001', '0036']
Get telescope state for device model Telecope Dish IDs ['0001', '0036']: sysytem under test TangoSUTDeployment{subarray_index=1; namespace=integration; tango_host=tango-databaseds.integration.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.integration.svc.miditf.internal.skao.int:8765}


Loading BokehJS ...

OK


#### 3.1.2 Open the inline dashboard
Start the simple inline dashboard showing current state of the Telescope and resource assignment and configuration status.

In [14]:
telescope_monitor_plot.show()
telescope_state.activate()
telescope_state.wait_til_ready(2)
print("OK")

OK


### 3.2 Print System Diagnostics

#### 3.2.1 Print TMC Diagnostics

In [25]:
sut.print_tmc_diagnostics()
print("OK")

TMC Central Node state: ON
TMC Central Node adminMode: AdminMode.OFFLINE
TMC Central Node healthState: HealthState.OK
TMC Central Node telescopeHealthState: HealthState.UNKNOWN
TMC Central Node isDishVccConfig: True
TMC Central Node dishvccvalidationstatus: {"dish": "ALL DISH OK", "ska_mid/tm_leaf_node/csp_master": "TMC and CSP Master Dish Vcc Version is Same"}
TMC Subarray Node state: ON
TMC Subarray adminMode: AdminMode.OFFLINE
TMC Subarray Node obsState: ObsState.EMPTY
OK


#### 3.2.2 Print CSP-LMC Diagnostics

In [16]:
sut.print_csp_diagnostics()
print("OK")

CSP-LMC Controller adminMode: AdminMode.OFFLINE
CSP-LMC Controller State: DISABLE
CSP-LMC Controller dishVccConfig: 
CSP-LMC Controller CBFSimulationMode: True
CSP-LMC Subarray adminMode: AdminMode.OFFLINE
CSP-LMC Subarray State: DISABLE
CSP-LMC Subarray obsState: ObsState.EMPTY
CSP-LMC Subarray dishVccConfig: 
OK


#### 3.2.3 Print CBF Diagnostics

In [17]:
sut.print_cbf_diagnostics()
print("OK")

CBF Controller adminMode: AdminMode.OFFLINE
CBF Controller State: DISABLE
CBF Subarray adminMode: AdminMode.OFFLINE
CBF Subarray State: DISABLE
CBF Subarray obsState: ObsState.EMPTY
OK


#### 3.2.4 Print SDP Diagnostics

In [18]:
sut.print_sdp_diagnostics()
print("OK")

SDP Controller state: STANDBY
SDP Controller adminMode: AdminMode.ONLINE
SDP Subarray state: OFF
SDP Subarray adminMode: AdminMode.ONLINE
SDP Subarray obsState: ObsState.EMPTY
OK


#### 3.2.5 Print Dish-LMC Diagnostics

In [19]:
for dish_deployment in dish_deployments:
    print(f"Dish {dish_deployment.dish_id} - {dish_deployment.namespace}: Diagnostics")

    dish_deployment.print_diagnostics()
print("OK")

Dish ska001 - integration-dish-lmc-ska001: Diagnostics
ska001: ComponentStates: "{'SPF': {'operatingmode': <SPFOperatingMode.STANDBY_LP: 2>, 'powerstate': <SPFPowerState.LOW_POWER: 1>, 'healthstate': <HealthState.UNKNOWN: 3>, 'bandinfocus': <BandInFocus.UNKNOWN: 0>, 'b1capabilitystate': <SPFCapabilityStates.STANDBY: 1>, 'b2capabilitystate': <SPFCapabilityStates.STANDBY: 1>, 'b3capabilitystate': <SPFCapabilityStates.STANDBY: 1>, 'b4capabilitystate': <SPFCapabilityStates.STANDBY: 1>, 'b5acapabilitystate': <SPFCapabilityStates.STANDBY: 1>, 'b5bcapabilitystate': <SPFCapabilityStates.STANDBY: 1>}, 'DS': {'healthstate': <HealthState.UNKNOWN: 3>, 'operatingmode': <DSOperatingMode.STANDBY_LP: 2>, 'pointingstate': <PointingState.READY: 0>, 'achievedtargetlock': None, 'indexerposition': <IndexerPosition.B2: 2>, 'powerstate': <DSPowerState.LOW_POWER: 3>, 'desiredpointingaz': [1715349400.0, 0.0], 'desiredpointingel': [1715349400.0, 0.0], 'achievedpointing': [1715349400.0, 0.0, 85.0], 'achievedpoin

#### 3.2.6 Print Full System Diagnostics

In [20]:
print("SUT: Full Diagnostics")
sut.print_full_diagnostics()

for dish_deployment in dish_deployments:
    print(f"Dish {dish_deployment.dish_id}: Full Diagnostics")
    dish_deployment.print_full_diagnostics()

print("Test Equipment: Full Diagnostics")
test_equipment.print_full_diagnostics()
print("OK")

SUT: Full Diagnostics
integration: ska-tango-base: sys/tg_test/1:

{
    "name": "sys/tg_test/1",
    "deployment_status": "Running",
    "device_properties": {},
    "device_attributes": [
        "ampli",
        "boolean_scalar",
        "double_scalar",
        "double_scalar_rww",
        "double_scalar_w",
        "float_scalar",
        "long64_scalar",
        "long_scalar",
        "long_scalar_rww",
        "long_scalar_w",
        "no_value",
        "short_scalar",
        "short_scalar_ro",
        "short_scalar_rww",
        "short_scalar_w",
        "string_scalar",
        "throw_exception",
        "uchar_scalar",
        "ulong64_scalar",
        "ushort_scalar",
        "ulong_scalar",
        "enum_scalar",
        "freq",
        "enum_scalar_ro",
        "boolean_spectrum",
        "boolean_spectrum_ro",
        "double_spectrum",
        "double_spectrum_ro",
        "float_spectrum",
        "float_spectrum_ro",
        "long64_spectrum_ro",
        "long_spectr

integration: ska-csp-lmc-mid: mid-csp/subarray/03:

{
    "name": "mid-csp/subarray/03",
    "deployment_status": "Running",
    "device_properties": {
        "ConnectionTimeout": [
            "900"
        ],
        "PingConnectionTime": [
            "5"
        ],
        "DefaultCommandTimeout": [
            "5"
        ],
        "CbfSubarray": [
            "mid_csp_cbf/sub_elt/subarray_03"
        ],
        "CspController": [
            "mid-csp/control/0"
        ],
        "SkaLevel": [
            "2"
        ],
        "SubID": [
            "3"
        ],
        "SubarrayProcModeCorrelation": [
            "mid_csp/elt/correlation-03"
        ],
        "SubarrayProcModeVlbi": [
            "mid_csp/elt/vlbi-03"
        ]
    },
    "device_attributes": [
        "buildState",
        "versionId",
        "loggingLevel",
        "loggingTargets",
        "healthState",
        "adminMode",
        "controlMode",
        "simulationMode",
        "testMode",
        "

integration: ska-mid-cbf-mcs: mid_csp_cbf/vcc_sw2/008:

{
    "name": "mid_csp_cbf/vcc_sw2/008",
    "deployment_status": "Unknown",
    "device_properties": {},
    "device_attributes": [
        "buildState",
        "versionId",
        "loggingLevel",
        "loggingTargets",
        "healthState",
        "adminMode",
        "controlMode",
        "simulationMode",
        "testMode",
        "obsState",
        "obsMode",
        "configurationProgress",
        "configurationDelayExpected",
        "activationTime",
        "configuredInstances",
        "usedComponents",
        "searchWindowTuning",
        "tdcEnable",
        "tdcNumBits",
        "tdcPeriodBeforeEpoch",
        "tdcPeriodAfterEpoch",
        "tdcDestinationAddress",
        "State",
        "Status"
    ],
    "state": "DISABLE",
    "chart": "ska-mid-cbf-mcs"
}
integration: ska-mid-cbf-mcs: mid_csp_cbf/fs_links/009:

{
    "name": "mid_csp_cbf/fs_links/009",
    "deployment_status": "Running",
    "devic

integration-dish-lmc-ska001: ska-tango-base: sys/tg_test/1:

{
    "name": "sys/tg_test/1",
    "deployment_status": "Running",
    "device_properties": {},
    "device_attributes": [
        "ampli",
        "boolean_scalar",
        "double_scalar",
        "double_scalar_rww",
        "double_scalar_w",
        "float_scalar",
        "long64_scalar",
        "long_scalar",
        "long_scalar_rww",
        "long_scalar_w",
        "no_value",
        "short_scalar",
        "short_scalar_ro",
        "short_scalar_rww",
        "short_scalar_w",
        "string_scalar",
        "throw_exception",
        "uchar_scalar",
        "ulong64_scalar",
        "ushort_scalar",
        "ulong_scalar",
        "enum_scalar",
        "freq",
        "enum_scalar_ro",
        "boolean_spectrum",
        "boolean_spectrum_ro",
        "double_spectrum",
        "double_spectrum_ro",
        "float_spectrum",
        "float_spectrum_ro",
        "long64_spectrum_ro",
        "long_spectrum",
 

integration-dish-lmc-ska036: ska-tango-base: sys/tg_test/1:

{
    "name": "sys/tg_test/1",
    "deployment_status": "Running",
    "device_properties": {},
    "device_attributes": [
        "ampli",
        "boolean_scalar",
        "double_scalar",
        "double_scalar_rww",
        "double_scalar_w",
        "float_scalar",
        "long64_scalar",
        "long_scalar",
        "long_scalar_rww",
        "long_scalar_w",
        "no_value",
        "short_scalar",
        "short_scalar_ro",
        "short_scalar_rww",
        "short_scalar_w",
        "string_scalar",
        "throw_exception",
        "uchar_scalar",
        "ulong64_scalar",
        "ushort_scalar",
        "ulong_scalar",
        "enum_scalar",
        "freq",
        "enum_scalar_ro",
        "boolean_spectrum",
        "boolean_spectrum_ro",
        "double_spectrum",
        "double_spectrum_ro",
        "float_spectrum",
        "float_spectrum_ro",
        "long64_spectrum_ro",
        "long_spectrum",
 

test-equipment: ska-tango-base: sys/tg_test/1:

{
    "name": "sys/tg_test/1",
    "deployment_status": "Running",
    "device_properties": {},
    "device_attributes": [
        "ampli",
        "boolean_scalar",
        "double_scalar",
        "double_scalar_rww",
        "double_scalar_w",
        "float_scalar",
        "long64_scalar",
        "long_scalar",
        "long_scalar_rww",
        "long_scalar_w",
        "no_value",
        "short_scalar",
        "short_scalar_ro",
        "short_scalar_rww",
        "short_scalar_w",
        "string_scalar",
        "throw_exception",
        "uchar_scalar",
        "ulong64_scalar",
        "ushort_scalar",
        "ulong_scalar",
        "enum_scalar",
        "freq",
        "enum_scalar_ro",
        "boolean_spectrum",
        "boolean_spectrum_ro",
        "double_spectrum",
        "double_spectrum_ro",
        "float_spectrum",
        "float_spectrum_ro",
        "long64_spectrum_ro",
        "long_spectrum",
        "long_

### 3.3 Setup ODA

In [21]:
os.environ["ODA_URI"] = (
    "http://ingress-nginx-controller-lb-default.ingress-nginx.svc.miditf.internal.skao.int/ska-db-oda/api/v1/"
)
eb_id = oda_helper.create_eb()
print(f"Execution Block ID: {eb_id}")
print("OK")

Execution Block ID: eb-miditf-20240510-00004
OK


### 3.4 Initialise Telescope and Subarray
Create Subarray and Telescope instances.

In [22]:
sub = SubArray(subarray_id)
tel = Telescope()
print("OK")

OK


### 3.5 Load Dish-VCC Configuration in TMC

In [24]:
# This should only be executed for a fresh deployment (i.e. Telescope is OFF.
# If you have restarted the subarray, you should not run this command
sut.load_dish_vcc_config()
print("OK")

CSP Controller: adminMode=0; State=OFF
CSP Controller: adminMode=0; State=OFF after 0s.
CSP Controller: adminMode=0; State=OFF
TMC Central Node isDishVccConfigSet=True after 0s
CSP Controller: adminMode=0; State=OFF
TMC Central Node: isDishVccConfigSet=True; dishvccvalidationstatus={"dish": "ALL DISH OK", "ska_mid/tm_leaf_node/csp_master": "TMC and CSP Master Dish Vcc Version is Same"}
TMC CSP Master Leaf Node: sourceDishVccConfig={"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "tm_data_sources": ["car://gitlab.com/ska-telescope/ska-telmodel-data?ska-sdp-tmlite-repository-1.0.0#tmdata"], "tm_data_filepath": "instrument/ska1_mid_psi/ska-mid-cbf-system-parameters.json"}; dishVccConfig={"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 119}, "SKA036": {"vcc": 2, "k": 1127}, "SKA063": {"vcc": 3, "k": 620}, "SKA100": {"vcc": 4, "k": 101}}}
OK


### 3.6 Turn telescope ON

In [26]:
# set to ON only if OFF
# If you have restarted the subarray, you should not run this command (Telescope is already ON)
# dish_lmc mode must be in LP_standby and before trying to turn the telescope ON
# Takes about 1m20s
if telescope_monitor_plot.on_off_state == "OFF":  # e.g. purple
    tel.on()
else:
    assert (
        telescope_monitor_plot.on_off_state == "ON"
    ), f"Cant continue with telescope in {telescope_monitor_plot.on_off_state}"
print("OK")

1|2024-05-10T15:21:30.554Z|DEBUG|MainThread|wrapper|oda_helper.py#244||Capturing the request function telescope_on in an ExecutionBlock
1|2024-05-10T15:21:30.555Z|INFO|MainThread|_call_and_wait_for_transition|common.py#147||Using pub/sub to track telescopeState of ska_mid/tm_central/central_node
1|2024-05-10T15:21:30.555Z|DEBUG|MainThread|subscribe_event|tango_executor.py#136||Observing ska_mid/tm_central/central_node/telescopeState
1|2024-05-10T15:21:30.623Z|DEBUG|MainThread|_subscribe|tango_executor.py#423||Subscribing to ska_mid/tm_central/central_node/telescopeState
1|2024-05-10T15:21:30.671Z|DEBUG|MainThread|notify_observers|tango_executor.py#359||Discarding first event: EventData[
     attr_name = 'tango://tango-databaseds.integration.svc.miditf.internal.skao.int:10000/ska_mid/tm_central/central_node/telescopestate'
    attr_value = DeviceAttribute(data_format = tango._tango.AttrDataFormat.SCALAR, dim_x = 1, dim_y = 0, has_failed = False, is_empty = False, name = 'telescopeState'

### 3.7 Observation Definition

#### 3.7.1 Define Resources to be used during Observation
Generate Processing Block and Execution Block IDs using SKUID.

In [27]:
# External IDs (this would typically come from a database like ODA)
# this is simulated by means of instantiate a new Observation object comping from the helper modules
observation = ObservationSB()
print("OK")

OK


#### 3.7.2 Create the high level observation specifications in terms of target specs

Note :- Users may currently modify the values by replacing the example values as given for each field within Target specification section.

In [28]:
observation = ObservationSB()
dish_ids = [d.dish_id.upper() for d in dish_deployments]
DEFAULT_TARGET_SPECS = {
    "flux calibrator": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="flux calibrator",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels9",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
    "M87": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="M87",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels10",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
}


channel_configuration = [
    Channel(
        spectral_window_id="fsp_1_channels",
        count=14880,
        start=0,
        stride=2,
        freq_min=0.35e9,
        freq_max=0.368e9,
        link_map=[[0, 0], [200, 1], [744, 2], [944, 3]],
    )
]

for key, value in DEFAULT_TARGET_SPECS.items():
    observation.add_channel_configuration(value.channelisation, channel_configuration)

observation.add_target_specs(DEFAULT_TARGET_SPECS)

for target_id, target in DEFAULT_TARGET_SPECS.items():
    observation.add_scan_type_configuration(
        config_name=target_id,
        beams={"vis0": BeamMapping(beam_id="vis0", field_id="M83")},
        derive_from=".default",
    )
scan_def_id = "flux calibrator"
observation.add_scan_sequence([scan_def_id])
print("OK")

OK


#### 3.7.3 Mid configuration schema input used by observing commands

[Configuration Schemas-OET→TMC(Mid)](https://developer.skao.int/projects/ska-telmodel/en/latest/)

In [29]:
telescope_monitor_plot.show()
print("OK")

OK


#### 3.7.4 Create Scheduling Block Definition(SBD) Instance and save it into the ODA

In [30]:
observation.eb_id = eb_id
pdm_allocation = observation.generate_pdm_object_for_sbd_save(DEFAULT_TARGET_SPECS)

sbd = oda_helper.save(pdm_allocation)
sbd_id = sbd.sbd_id
pdm_allocation.sbd_id = sbd_id
print(f"Saved Scheduling Block Definition Instance in ODA: SBD_ID={sbd_id}")
print("OK")

1|2024-05-10T15:22:25.098Z|DEBUG|MainThread|save|oda_helper.py#66||Sending request to http://ingress-nginx-controller-lb-default.ingress-nginx.svc.miditf.internal.skao.int/ska-db-oda/api/v1/sbds
Saved Scheduling Block Definition Instance in ODA: SBD_ID=sbd-miditf-20240510-00005
OK


### 3.8 Assign Resources
Assign the requested resources to a Subarray

In [32]:
print("Assign request")
assign_request = observation.generate_allocate_config_sb(pdm_allocation).as_object
print(f"Got assign request {assign_request}")
print()

if debug_mode:
    print("Debug request JSON")
    request_json = get_request_json(assign_request, AssignResourcesRequest, True)
    print("AssignResourcesRequest:", json.dumps(json.loads(request_json), indent=2))
print()

print("Assign from Control Data Model (CDM) from subarray {sub.id}")
try:
    sub.assign_from_cdm(assign_request, timeout=120)
except tango.DevFailed as terr:
    print(f"ERROR: {terr.args[0].desc.strip()}")
except EventTimeoutError:
    print("Could not assign resources")
print()

print("Plot telescope monitor")
try:
    telescope_monitor_plot.show()
except tango.DevFailed as terr:
    print(f"ERROR: {terr.args[0].desc.strip()}")
print()

print("OK")

Assign request
1|2024-05-10T15:28:25.203Z|INFO|MainThread|convert_channels|sdp.py#290||Setting ChannelConfiguration id:vis_channels10 , spectral_windows:[<SpectralWindow(spectral_window_id=fsp_1_channels, count=14880, start=0, stride=2, freq_min=350000000.0, freq_max=368000000.0, link_map=[(0, 0), (200, 1), (744, 2), (944, 3)])>] 
1|2024-05-10T15:28:25.206Z|INFO|MainThread|convert_spectral_window|sdp.py#269||Setting channel attribute -> count:14880 , start:0 , stride:2, freq_min:350000000.0, freq_max:368000000.0 , link_map:[(0, 0), (200, 1), (744, 2), (944, 3)] 
1|2024-05-10T15:28:25.207Z|INFO|MainThread|convert_channels|sdp.py#290||Setting ChannelConfiguration id:vis_channels9 , spectral_windows:[<SpectralWindow(spectral_window_id=fsp_1_channels, count=14880, start=0, stride=2, freq_min=350000000.0, freq_max=368000000.0, link_map=[(0, 0), (200, 1), (744, 2), (944, 3)])>] 
1|2024-05-10T15:28:25.208Z|INFO|MainThread|convert_spectral_window|sdp.py#269||Setting channel attribute -> count:

1|2024-05-10T15:28:25.786Z|DEBUG|MainThread|_open_nexus|backend.py#766||Querying Nexus: GET https://artefact.skao.int/repository/raw-telmodel/gitlab.com/ska-telescope/ska-telmodel/tmdata/1.13.0
1|2024-05-10T15:28:26.321Z|DEBUG|MainThread|_open_nexus|backend.py#766||Querying Nexus: GET https://artefact.skao.int/repository/raw-telmodel/gitlab.com/ska-telescope/ska-telmodel/tmdata/1.13.0
1|2024-05-10T15:28:26.874Z|DEBUG|MainThread|_open_nexus|backend.py#766||Querying Nexus: GET https://artefact.skao.int/repository/raw-telmodel/gitlab.com/ska-telescope/ska-telmodel/tmdata/1.13.0
1|2024-05-10T15:28:27.407Z|INFO|MainThread|_call_and_wait_for_transition|common.py#147||Using pub/sub to track obsState of ska_mid/tm_subarray_node/1
1|2024-05-10T15:28:27.408Z|DEBUG|MainThread|subscribe_event|tango_executor.py#136||Observing ska_mid/tm_subarray_node/1/obsState
1|2024-05-10T15:28:27.409Z|INFO|MainThread|_get_id_from_params_or_generate_new_id|transactions.py#172||Generated transaction ID txn-local-2

1|2024-05-10T15:28:27.467Z|INFO|MainThread|log_exit|transactions.py#150||Transaction[txn-local-20240510-534523023]: Exit[AssignResources] marker[31848]
1|2024-05-10T15:28:27.467Z|DEBUG|MainThread|unsubscribe_event|tango_executor.py#154||Unobserving ska_mid/tm_subarray_node/1/obsState
1|2024-05-10T15:28:27.468Z|DEBUG|MainThread|send_request_response|oda_helper.py#218||Sending request to http://ingress-nginx-controller-lb-default.ingress-nginx.svc.miditf.internal.skao.int/ska-db-oda/api/v1/ebs/eb-miditf-20240510-00004/request_response with body {'error': {'detail': 'DevFailed(args = (DevError(desc = '
                     "'ska_tango_base.faults.StateModelError: AssignResources "
                     "command not permitted in observation state 1\\n', origin "
                     "= 'Traceback (most recent call last):\\n  File "
                     '"/usr/local/lib/python3.10/dist-packages/tango/server.py", '
                     'line 1508, in cmd\\n    return get_worker().execute(f, '


OK


In [ ]:
telescope_monitor_plot.show()
print("OK")

### 3.9 Configure Scan
Configure the telescope  on first target in sequence - may be modified to configure and run multiple targets at a later time.

In [ ]:
configure_object = observation.generate_scan_config_sb(
    pdm_observation_request=pdm_allocation,
    scan_definition_id=scan_def_id,
    scan_duration=10.0,
).as_object

if debug_mode:
    cfg_json = get_request_json(configure_object, ConfigureRequest)
    print(f"ConfigureRequest={cfg_json}")

sub.configure_from_cdm(configure_object, timeout=120)
time.sleep(2)
print("OK")

In [ ]:
telescope_monitor_plot.show()
print("OK")

### 3.10 Run the Scan

In [ ]:
sub.scan(timeout=120)
telescope_monitor_plot.show()
print("OK")

### 3.11 Post Observation teardown
If the observation executed successfully, you can use the following commands to reset the telescope.

#### 3.11.1 Clear scan configuration 

In [ ]:
sub.end()
telescope_monitor_plot.show()
print("OK")

#### 3.11.2 Release Subarray resources

In [ ]:
sub.release()
telescope_monitor_plot.show()
print("OK")

### 3.12 Reset the Telescope/Subarray (On Failure)
Set booleans to True to reset the system after a failed execution.

In [ ]:
do_reset_subarray = False
do_reset_dish = False

if do_reset_subarray:
    sub.abort()
    time.sleep(3)
    sub.restart()

if do_reset_dish:
    dish_deployment: TangoDishDeployment
    for dish_deployment in dish_deployments:
        dish_deployment.reset_dish()
print("OK")

## 4 Debug Telescope

### 4.1 Display Telescope State

In [ ]:
telescope_state.state
print("OK")

### 4.2 Display Telescope State Subscriptions

In [ ]:
telescope_state.state_monitor.subscriptions
print("OK")

### 4.3 Display Dish LMC State

In [ ]:
dishes_to_debug = [d.dish_id for d in dish_deployments]
for dish in dish_deployments:
    if dish.dish_id in dishes_to_debug:
        dish.print_diagnostics()
print("OK")

### 4.4 Display CSP LMC State

In [ ]:
sut.print_csp_diagnostics()
print("OK")

### 4.5 Display TMC State

In [ ]:
sut.print_tmc_diagnostics()
print("OK")

### 4.6 Display SDP State

In [ ]:
sut.print_sdp_diagnostics()
print("OK")

## 5 Dashboard Links

### 5.1 Taranta

In [ ]:
print(f"SUT: {sut.taranta_endpoint}")
print(f"Test Equipment: {test_equipment.taranta_endpoint}")
for dish in dish_deployments:
    print(f"Dish LMC {dish.dish_id}: {dish.taranta_endpoint}")
print("OK")

### 5.2 Data Product Dashboard

https://k8s.miditf.internal.skao.int/ska-dpd/dashboard/

### 5.3 SDP Signal Displays

In [ ]:
print(sut.signal_displays_endpoint)
print("OK")

## 6 System Health Displays Tests

### Copied from Jama 2024-03-28

### 6.1 Display the system status and bring the telescope into a known operating state of Standby (SPFC, Dish Structure Controller Simulator, SPFRx, CBF, SDP)

Running steps 0.1 and 2.1-2.6 should get the telescope into a standby state which can be checked by running 3.1 and 3.3.1

### 6.2 Use TMC and LMC dashboards to show status and sensor values from each subsystem. Temperature sensors should be displayed where available.
Run dashboards with links in step 4.2

### 6.3 Use the EDA configuration supplied by each product to configure the EDA in the ITF to subscribe to a range of relevant Tango attribute sensors from each product. Temperature sensors should be displayed where available.

### 6.4 Show that archived sensor values can be displayed or plotted over a configurable time period

### 6.5 Show that several historical sensor values can be plotted on the same axis.

### 6.6 Configure the alarm handler in the ITF to subscribe to a range of relevant Tango attribute sensors from each product.

### 6.7 Use alarm handler to display which sensors are currently in Warning and Alarm status.

### 6.8 Show the drill-down display from the top level system to the products in the system, showing sensor values and associated health and alarm displays

### 6.9 Force an alarm condition and verify that the alarm is reported with the correct timestamp by the alarm handler.  Repeat for various sensors. (it may be necessary to change the alarm threshold values to create the alarm condition)

### 6.10 Perform network interruption conditions (eg. remove a network cable for a period) to demonstrate communication loss alarm indications and recovery.

### 6.11 Display simulated pointing position of antennas and target information